In [42]:
from bs4 import BeautifulSoup
import requests
import copy
import re
import pickle
from collections import defaultdict
import sys
sys.setrecursionlimit(40000)

actors_list = []


# retrieve actors list
def get_actors_list(tag, style):
    dirty_actors_list = list(soup.findAll(tag, style=style))
    clean_actors_list = []
    for line in dirty_actors_list:
        clean_actors_list.append(line.a.string)
    return clean_actors_list

def get_bankability_metrics(tag, style):
    # primary bankability metrics________________________________________________
    dirty_bank_list = list(soup.findAll(tag, style=style))
    clean_bank_list = []
    for line in dirty_bank_list:
        clean_bank_list.append(int(line.text[1:].replace(',','')))
    # print(clean_bank_list) # test-print

    # secondary bankability metrics________________________________________________
    movs_per_yr_list = []
    bank_per_yr_list = []
    bank_per_mov_list = []
    for n in dirty_bank_list:
        string_container = n.nextSibling.nextSibling.get_text()
        # print(string_container, "END \n")  # temp
        # parse through loop and retrieve movies per year for each actor
        divider1 = string_container.find(" movies/year")
        movs_per_yr_list.append(float(string_container[2:divider1]))
        # parse through loop to retrieve bankability per year for each actor
        divider2 = string_container.find("/year", divider1) + divider1
        divider3 = string_container.find("/year", divider2 + divider1)
        bank_per_yr_list.append(int(string_container[divider2 + divider1:divider3].replace(',', '')))
        bank_per_mov_list.append(bank_per_yr_list[-1] / movs_per_yr_list[-1])
    
    # test-prints______________________________________
    # print(movs_per_yr_list, len(movs_per_yr_list))
    # print(bank_per_yr_list, len(bank_per_yr_list))
    # print(bank_per_mov_list, len(bank_per_mov_list))
    return clean_bank_list


# Primary Roles________________________________________________
def get_roles_in_films(identifier):
    films_dict = defaultdict(list)
    rank = 1
    # print('length = ', len(list(soup.find_all(id="col2mid")))) # temp
    for l in list(soup.find_all(id=identifier)):
        for r in list(re.findall(r'summary">(.*?)<', str(l))):
            # attempting to replace gibberish chars with appropriate apostrophes and colons
            if r.find('â') != -1:
                if r[r.find('â')+3] == 's':
                    r = r[:r.find('â')]+"'"+r[r.find('â')+3:]
                else:
                    r = r[:r.find('â')]+":"+r[r.find('â')+2:]
            # print('X94! ', r.find('x94')) # r.replace('\\x94','')
            if r not in films_dict[str(rank)]: # check if duplicate, skip if it is
                films_dict[str(rank)].append(r)
        rank +=1
    return films_dict

# MAIN CODE _____________________________________________________
# GET and prettify via Beautiful Soup
html_page = requests.get("https://www.the-numbers.com/bankability") #Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
# soup.prettify # test-print

actors_list = get_actors_list('span', "font-size:200%;")
bank_list = get_bankability_metrics('div', "font-size:200%;")
roles_in_films_dict = get_roles_in_films("col2mid")


# put it all together____________________________
main_dict = {}
attributes = {}
if len(actors_list) == len(bank_list):
    for i in range(len(actors_list)):
        attributes['rank'] = i+1
        attributes['bankability'] = bank_list[i]
        # attributes['bank per movie'] = bank_per_mov_list[i]
        attributes['roles'] = roles_in_films_dict[str(i+1)]
        main_dict[actors_list[i]] = attributes.copy()
else:
    raise ValueError
# main_dict # test-print


In [43]:
filename = "actors_dict"
outfile = open(filename, 'wb')

In [44]:
pickle.dump(main_dict, outfile)
outfile.close()

In [45]:
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()

In [46]:
# confirm it's loaded accurately
print(new_dict)
print(new_dict==main_dict)
print(type(new_dict))

{'Tom Cruise': {'rank': 1, 'bankability': 22537572, 'roles': ['Mission: Impossible:\x94Fallout', 'Mission: Impossible:\x94Rogue Nation', 'Mission: Impossible:\x94Ghost Protocol', 'War of the Worlds', 'Austin Powers in Goldmember', 'Mission: Impossible 2', 'Mission: Impossible III']}, 'Will Smith': {'rank': 2, 'bankability': 20593743, 'roles': ['Aladdin', 'Suicide Squad', 'I am Legend', 'Hancock', 'Men in Black 3', 'Hitch', 'The Pursuit of Happyness', 'Annie', 'I, Robot']}, 'Robert Downey, Jr.': {'rank': 3, 'bankability': 16602313, 'roles': ['Avengers: Endgame', 'Avengers: Infinity War', 'The Avengers', 'Avengers: Age of Ultron', 'Captain America: Civil War', 'The Judge', 'A Guide to Recognizing Your Saints']}, 'Sandra Bullock': {'rank': 4, 'bankability': 15694181, 'roles': ['Minions', 'Gravity', 'The Blind Side', "Ocean's 8", 'The Heat', 'The Proposal', 'Miss Congeniality', 'Two Weeks Notice', 'Miss Congeniality 2: Armed and Fabulous', 'Hope Floats']}, 'Kathleen Kennedy': {'rank': 5, '

In [ ]:
import config
import mysql.connector ## Connect to DB server on AWS

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = "Movie Project1"
)

cursor = cnx.cursor()
print(cnx)

In [ ]:
for actor in main_dict:
    for i in range(len(main_dict[actor]['roles'])):
        # INSERT INDIVIDUALLY
        # print(type(actor), type(main_dict[actor]['bankability']), type(main_dict[actor]['roles'][i]))
        add_actor_line = ("INSERT INTO Actors "
               "(name, bankability, title) "
               "VALUES (%s, %s, %s)")
        data_actor_line = (str(actor), main_dict[actor]['bankability'], main_dict[actor]['roles'][i])
        # Insert new actor line
        cursor.execute(add_actor_line, data_actor_line)

cnx.commit()

In [ ]:
query = ("SELECT DISTINCT name, bankability FROM Actors ")

# hire_start = datetime.date(1999, 1, 1)
# hire_end = datetime.date(1999, 12, 31)

cursor.execute(query)

for (first_name, last_name, hire_date) in cursor:
    print("{}, {} was hired on {:%d %b %Y}".format(
    last_name, first_name, hire_date))

In [ ]:
cursor.close()
cnx.close()